<a href="https://colab.research.google.com/github/hanbun-pom/siosio/blob/master/Yukapedia_Git%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

不特定多数への公開は今の所ご遠慮下さい  
当ページ外で行われた操作についての責任は負いかねます  
@hanbun_pom

---
####～使い方～  
① 検索条件を入力  
② ↓にカーソルを合わせると表示される ▶ を押して実行

In [24]:
import pandas as pd
import datetime

#@markdown ---
名前 = ""  #@param {type:"string"}
名前完全一致 = False  #@param {type:"boolean"} 
名前正規表現 = False  #@param {type:"boolean"} 
#@markdown ---
コメント = "^\u9EBB\u96C0\u30ED\u30DC\u30C3\u30C8$|^\u9EBB\u96C0\u30DE\u30B7\u30FC\u30F3$"  #@param {type:"string"}
コメント完全一致 = False  #@param {type:"boolean"}
コメント正規表現 = True  #@param {type:"boolean"} 
#@markdown ---
t秒前から再生 = 30 #@param {type:"slider", min:0, max:60, step:5}
#@markdown ---
配信日_以降 = '2019-12-24' #@param {type:"date"}
配信日_以前 = '2040-04-01' #@param {type:"date"}　


try:
    df

except NameError:

    # 配信一覧　取得
    youtube = 'https://youtu.be/'
    master = 'https://raw.githubusercontent.com/hanbun-pom/siosio/master/'
    videos = pd.read_csv(master + 'videos.csv')

    # コメント一覧　作成
    df_list=[]
    for Id in videos['videoId']:
        tmp_df = pd.read_table(master + Id + '.tsv', header=None, names=['time','name','comment','videoId'],quotechar='"', quoting=3)
        tmp_df['videoId']= Id
        df_list.append(tmp_df)
    df = pd.concat(df_list, ignore_index=True)

    # 配信タイトル、公開時間
    df = df.merge(videos)
    df['DateTime'] = pd.to_datetime(df['DateTime']).dt.date

    # 再生時間指定URL
    trans = str.maketrans({':': '',',': ''})
    s = df['time'].str.translate(trans).astype(int)
    s = s-40*(abs(s)//100)-2400*(abs(s)//10000)-t秒前から再生
    s = ((abs(s)+s)//2).astype(str)

    df['videoId'] = youtube + df['videoId'] + '?t=' + s + 's'

else:
    pass

#名前、コメント検索
def query_condition(data, col, query, is_perfect, is_regex):
    if is_perfect:
        return data[col].str.match(query) if is_regex else data[col] == query
    else:
        return data[col].str.contains(query, na=False, regex=is_regex)

name_condition = query_condition(df, "name", 名前, 名前完全一致, 名前正規表現)
comment_condition = query_condition(df, "comment", コメント, コメント完全一致, コメント正規表現)

df2 = df[name_condition & comment_condition].copy()

# 配信日指定
start = datetime.datetime.strptime(配信日_以降, '%Y-%m-%d').date()
end = datetime.datetime.strptime(配信日_以前, '%Y-%m-%d').date()
df3 = df2[df2['DateTime'] >= start]
df3 = df3[df3['DateTime'] <= end]

pd.set_option('display.max_rows', 10000)
df3

,time,name,comment,videoId,title,DateTime
60714,58:55,いえろ〜ず,麻雀マシーン,https://youtu.be/NvMIkTa9SXs?t=3505s,【雀魂】コンビ打ち麻雀🔥本当の&quot;お引き&quot;見せます【楠栞桜視点】,2020-04-03
189420,2:34:15,CarMania385,麻雀ロボット,https://youtu.be/eBCbqMZajKs?t=9225s,【パズル探偵スカウト】楠栞桜、今日から探偵業始めます。【Vtuber】,2020-03-14
218029,49:21,カラスは可愛いのですよ?,麻雀マシーン,https://youtu.be/oBVYwlBFwa0?t=2931s,【雀魂】可愛い女の子と麻雀ができるって聞きました #1【Vtuber】,2020-03-06
383054,2:52:42,ひろかわ,麻雀ロボット,https://youtu.be/imD90glgPdo?t=10332s,【天鳳】レート1800こえるまで終われません!!【Vtuber】,2020-02-21
471426,47:47,北傘,麻雀ロボット,https://youtu.be/2LRU2MLvy8E?t=2837s,【雀魂】天開司さんと視聴者参加型麻雀配信🀄【Vtuber】,2020-02-08
471444,47:51,月十字,麻雀ロボット,https://youtu.be/2LRU2MLvy8E?t=2841s,【雀魂】天開司さんと視聴者参加型麻雀配信🀄【Vtuber】,2020-02-08
631052,14:22,不座家ルナ,麻雀ロボット,https://youtu.be/WNP2ZU_7Nms?t=832s,【ボイド・テラリウム】今日発売のゲーム！！【Vtuber】,2020-01-23
631088,14:32,もるぽん,麻雀ロボット,https://youtu.be/WNP2ZU_7Nms?t=842s,【ボイド・テラリウム】今日発売のゲーム！！【Vtuber】,2020-01-23
789850,1:40:37,白音とと,麻雀ロボット,https://youtu.be/LZVdK5kgSOk?t=6007s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24


2020/03/19 追加　完全一致検索  正規表現検索　動画URL生成  
2020/04/07 変更　GitHub版  